This is my first attempt at building a CNN inspired by the LeNet architecture. The Lenet architecture consists of two sets of convolutional and pooling layers, followed by two fully connected layers. Let's get to building!

Before we get to building, we need to generate data. So far, I have generated data from the following datasets:
* i3c3a1d0v5a0_set18a
* i3c3a1d0v5a0_set18b
* i3c3a1d0v5a0_set18d
* i3c3a1d0v5a2_set21
* i3c3a1d0v5a7_set21
* i3c3a1d0v5a14_set21
* i3c3a1d0v5a15_set21
* i3c3a1d1v5a0_set22
* i3c3a1d1v5a7_set22
* i3c4a1d0v5a0_set20
* i3c4a1d0v5a0_set23
* i3c5a1d0v5a0_set20
* i3c5a1d0v5a0_set23
* i3c3a0d0v5a2_set17 - bad data set, ignore
* i3c3a0d1v5a2_set17 - also bad data set, ignore 
The two last datasets are deemed as "bad" because they did not have any T1 transitions. I suspect that something was weird with the data generation process from the master code, but I am not sure. 

Firstly, import the necessary libraries. These ones are pretty standard, courtesy of Harish's notebook for generating the dataset.

In [3]:
# from phasefield_scripts.load_files import *
# from phasefield_scripts.procedures import *
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import ipywidgets as widget
from ipywidgets import fixed
import os
import pickle
import multiprocessing as mp
from tqdm import tqdm
from multiprocessing import Pool
%matplotlib widget
import pandas as pd
import numpy as np 

Import Torch libraries to make building the CNN a lot easier.

In [4]:
import torch
import torchvision
from torchvision import transforms
# from torchvision.datasets import ImageFolder

c:\Users\Nigar\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Nigar\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


Now, time to import your dataset. Harish has already generated the dataset for me - both the images and numpy arrays. I will be importing the arrays directly, to then turn them into tensors using PyTorch. 

In [5]:
#train and test data directory
expname_train = 'i3c3a1d0v5a0_set18a'
expname_test = 'i3c3a1d0v5a0_set18b'

input_train_dir = 'data\\' + expname_train
data_train_dir = input_train_dir + '/images'

input_test_dir = 'data\\' + expname_test
data_test_dir = input_test_dir + '/images'

length_tissue = len([name for name in os.listdir(data_train_dir) if name.startswith('tissue')])
print(length_tissue)


test = np.load(data_train_dir + '\\tissue00000.npy')
print(test)
print(test.shape)

50
[[False False  True ... False False False]
 [False False  True ... False False False]
 [False False  True ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
(40, 40)


So here, I have noticed that it is not as trivial to make an array of arrays... So I tried importing images instead of the arrays in hopes of making my life easiert, but it did not really help - ImageFolder from PyTorch wants me to sort my images into folders (while also implementing classes for T1 == yes and T1 == no), which I have not done yet. In order to sort my files the way PyTorch wants me to, I will use my T1-transition arrays and make the correct folders. But this is something for future Nigar to worry about tomorrow... :) 

In [17]:
# images = data_train_dir + '/img'
import os
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# define the path to the directory containing the image data
data_dir = data_train_dir

# define the image transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

# create a dataset object using the ImageFolder class
dataset = ImageFolder(root=data_dir, transform=transform)

# create a dataloader object using the DataLoader class
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# iterate over the batches of images
for images, labels in dataloader:
    # do something with the images and labels
    print(images.shape, labels.shape)



FileNotFoundError: Couldn't find any class folder in data\i3c3a1d0v5a0_set18a/images.

In [15]:
def data_loader(data_dir):
    tissue_train = []
    tissue_files = [file_name for file_name in os.listdir(data_dir) if file_name.startswith('tissue')]
    for file_name in tissue_files:
        tissue_train.append(file_name)
    data = np.array(tissue_train)
    return data

train_data = data_loader(data_train_dir)
test_data = data_loader(data_test_dir)


[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 0 1 0 0]


Small piece of code for counting T1 transitions in training set + test set. 

In [7]:
#open the T1 locations numpy file from images folder 
T1_labels_test = np.load(data_train_dir + '\\T1_status.npy') #18
T1_labels_train = np.load(data_test_dir + '\\T1_status.npy')

print(T1_labels_test)
print(T1_labels_train)

#function for counting the number of 1's in T1_status, sanity check 
def T1_true(T1_labels):
    T1_true = np.count_nonzero(T1_labels)
    return T1_true

print(T1_true(T1_labels_train))
print(T1_true(T1_labels_test))

7
6


Here, I was trying to reshape my arrays but it went terribly wrong...

In [51]:

#reshape the data to be 4D tensor
#this approach does not work!!!!
train_data = train_data.reshape(length_tissue, 1, 150, 150)
test_data = test_data.reshape(length_tissue, 1, 150, 150)




ValueError: cannot reshape array of size 50 into shape (50,1,150,150)

Now, most of the code above can be ignored. Tomorrow's Nigar has to find a way to sort her data into proper folders so that ImageFolder can be used. To do this, she will need to use T1_labels arrays for each experiment. Good luck :) 